In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# Load the ASL Alphabet dataset
data_dir = 'archive/asl_alphabet_train/asl_alphabet_train'  # Replace with the path to your dataset
classes = os.listdir(data_dir)
num_classes = len(classes)

# Initialize lists to store images and their labels
images = []
labels = []

# Load images and labels from the dataset
for class_idx, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Ensure color images
        image = cv2.resize(image, (50, 50))  # Resize the image to a consistent size
        images.append(image)
        labels.append(class_idx)

# Convert lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Load pre-trained VGG19 model without top layers (include_top=False)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(50, 50, 3))

# Add your own classification layers on top of VGG19
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),  # You can change the number of units as needed
    Dense(num_classes, activation='softmax')
])

# Freeze the convolutional layers of VGG19 (optional)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Saving h5 file:
model.save('FinalModel_TransferLearning_VGG19.h5')


80134624/80134624 [==============================] - 9s 0us/step
Epoch 1/10
1958/1958 [==============================] - 356s 181ms/step - loss: 1.0152 - accuracy: 0.7277 - val_loss: 0.4793 - val_accuracy: 0.8664
Epoch 2/10
1958/1958 [==============================] - 349s 178ms/step - loss: 0.3426 - accuracy: 0.9043 - val_loss: 0.2662 - val_accuracy: 0.9220
Epoch 3/10
1958/1958 [==============================] - 347s 177ms/step - loss: 0.2097 - accuracy: 0.9410 - val_loss: 0.1659 - val_accuracy: 0.9529
Epoch 4/10
1958/1958 [==============================] - 348s 178ms/step - loss: 0.1447 - accuracy: 0.9580 - val_loss: 0.1272 - val_accuracy: 0.9611
Epoch 5/10
1958/1958 [==============================] - 348s 178ms/step - loss: 0.1079 - accuracy: 0.9680 - val_loss: 0.1173 - val_accuracy: 0.9619
Epoch 6/10
1958/1958 [==============================] - 364s 186ms/step - loss: 0.0826 - accuracy: 0.9756 - val_loss: 0.0934 - val_accuracy: 0.9697
Epoch 7/10
1958/1958 [=========================